## VGG(Visual Geometry Group)
原始论文：《Very Deep Convolutional Networks For Large-Scale Image Recognition》

论文亮点：
+ 卷积核全部使用的是3 * 3的卷积核
+ 输入图像为224 * 224的RGB图像
+ 预处理：RGB三个通道减去各自通道的均值
+ 每一个卷积层后连接着三个全连接层（Fully-Connected layers，FC）。前两个全连接层各有4096个通道，第三个全连接层是1000种分类。最后一层的激活函数是softmax，其他层的激活函数是ReLU。
+ VGG有六种结构，每一种结构一般以VGG+层数命名。每一种结构具有不同的参数个数。

![VGG](https://cdn-images-1.medium.com/max/800/0*HREIJ1hjF7z4y9Dd.jpg)

+ 训练：使用mini-batch gradient descent with momentum。batch-size是256，momentum是0.9。使用了dropout（radio为0.5）。学习速率初始化为$10^{-2}$。衰减系数为10。
+ 总共训练了370K iterations，74 epochs.
+ 由于层数深的网络很难收敛，因此先训练层数少的网络直至收敛（如configuration A）。然后将A训练好的参数作为更深网络的参数输入进行训练。输入时只将网络A的前四个卷积层和最后三个全连接层作为参数输入，其他的参数仍然进行随机初始化。
+ 随机初始化的均值为0，方差为$10^{-2}$。bias的初始化值为0。

In [1]:
import tensorflow as tf
import numpy as np

#### tf.get_variable(name, shape, initializer, trainable)
`name`是变量的名称，`shape`是变量的维度，`initializer`是初始化函数。`trainable`如果为 True，还将变量添加到图形集`GraphKeys.TRAINABLE_VARIABLES`。一般`initializer`初始化的方式有以下几种：

+ tf.constant_initializer：常量初始化函数
+ tf.random_normal_initializer：随机正态分布
+ tf.truncated_normal_initializer：截取的正态分布
+ tf.random_uniform_initializer：均匀分布
+ tf.zeros_initializer：全都是0
+ tf.ones_initializer：全都是1
+ tf.uniform_unit_scaling_initializer：满足均匀分布，但不影响输出数量级的随机值

当然，tf.contrib.layers里还提供了一些其他的方法，例如这里使用的`tf.contrib.layer.xavier_initializer(uniform=True, seed=None, dtype=tf.float32)`。
该函数返回一个用于初始化权重的初始化程序`Xavier`，这个初始化器可以保持每一层的梯度大小都差不多相同。

参数：

    + uniform：使用uniform或者normal分布随机初始化
    + seed：用来生成随机数的seed
    + dtype：只支持浮点数
    
返回值：
    初始化权重矩阵

In [2]:
def conv(layer_name, x, out_channels, kernel_size=[3,3], stride=[1,1,1,1], is_pretrain=True):
    '''Convolution on wrapper, use ReLU activation after convolution
    Args:
        layer_name: e.g, conv1, poo1, ...
        x: input tensor, [batch_size, height, width, channels]
        out_channels: number of output channels(or convolutional kernels)
        kernel_size: the size of convolutional kernel, VGG paper used: [3, 3]
        stride: A list of ints. 1-D of length 4. VGG paper used: [1, 1, 1, 1]
    Returns:
        4D tensor
    '''
    # x.get_shape()得到x tensor的尺寸（返回一个list），最后一个元素是x的通道数
    in_channels = x.get_shape()[-1]
    with tf.variable.scope(layer_name):
        w = tf.get_variable(name='weights',
                           trainable=is_pretrain,
                           shape=[kernel_size[0], kernel_size[1], in_channels, out_channels],
                           initializer=tf.contrib.layers.xavier_initializer())
        b = tf.get_variable(name='biases',
                           trainable=is_pretrain,
                           shape=[out_channels],
                           initializer=tf.constant_initializer(0, 0))
        x = tf.nn.conv2d(x, w, stride, padding='SAME', name='conv')
        x = tf.nn.bias_add(x, b, name='bias_add')
        x = tf.nn.relu(x, name='relu')
        return x